# Modeling and Evaluation

In [7]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline

## Modeling

In [4]:
# read in data from csv with cluster labels
songs= pd.read_csv("../data/spotify_songs_clustered.csv")
songs.head()

,song,artist,album,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,time_signature,kmeans_cluster,dbscan_cluster
0,...Ready For It?,Taylor Swift,...Ready For It?,2017-09-03,0.615,0.779,2,-6.454,1,0.1350,0.0665,0.000000,0.1550,0.453,160.000,208198,4,7,0
1,Life Changes,Thomas Rhett,Life Changes,2017-09-08,0.687,0.845,7,-4.370,1,0.0576,0.1000,0.000000,0.0452,0.809,87.972,190227,4,0,0
2,24K Magic,Bruno Mars,24K Magic,2016-11-17,0.818,0.803,1,-4.282,1,0.0797,0.0340,0.000000,0.1530,0.632,106.970,225983,4,0,0
3,Galway Girl,Ed Sheeran,÷ (Deluxe),2017-03-03,0.624,0.876,9,-3.374,1,0.1000,0.0735,0.000000,0.3270,0.781,99.943,170827,4,2,0
4,Photograph,Ed Sheeran,x (Deluxe Edition),2014-06-21,0.614,0.379,4,-10.480,1,0.0476,0.6070,0.000464,0.0986,0.201,107.989,258987,4,3,0
